# Kerchunk

This is a companion notebook to [Kerchunk and Pangeo-Forge](notebooks/kerchunk-pangeo-forge.ipynb) to demonstrate how to recreate the processing pipeline using `Kerchunk` without `Pangeo-Forge`. Details on the dataset and processing steps are outlined in more detail in the previous notebook.

More detailed guides to `Kerchunk` can be found in the [Docs](https://fsspec.github.io/kerchunk/) and the [Project Pythia Cookbook](https://projectpythia.org/kerchunk-cookbook/README.html)

In [ ]:
import dask
import fsspec
import glob
import logging
import ujson
import warnings
import xarray as xr

from tempfile import TemporaryDirectory
from distributed import Client
from kerchunk.combine import MultiZarrToZarr
from kerchunk.hdf import SingleHdf5ToZarr

## Exaimine a Single File

In [ ]:
import xarray as xr

ds = xr.open_dataset(
    "http://thredds.northwestknowledge.net:8080/thredds/dodsC/MET/bi/bi_2021.nc"
)

In [ ]:
ds

### Startup a Dask Client for Parallel Processing

In [ ]:
client = Client(n_workers=8, silence_logs=logging.ERROR)
client

### Create List of URLs

In [ ]:
start_year = 1979
end_year = 1981  # Data is available until 2022. This example only processes 2 years

years = list(range(start_year, end_year + 1))

url_list = [
    f"http://www.northwestknowledge.net/metdata/data/bi_{time}.nc" for time in years
]

### Create a Temp Directory to Write

In [ ]:
import os
from tempfile import TemporaryDirectory

td = TemporaryDirectory()
target_root = td.name
target_root = "esip"
store_name = "kerchunk"
target_store = os.path.join(target_root, store_name)

### Use `SingleHdf5ToZarr` to Create Kerchunk References


In [ ]:
def generate_json_reference(fil, output_dir: str):
    h5chunks = SingleHdf5ToZarr(fil, inline_threshold=300)
    fname = fil.split("/")[-1].strip(".nc")
    outf = f"{output_dir}/{fname}.json"
    with open(outf, "wb") as f:
        f.write(ujson.dumps(h5chunks.translate()).encode())
    return outf


# Generate Dask Delayed objects
tasks = [dask.delayed(generate_json_reference)(fil, target_store) for fil in url_list]

In [ ]:

%%time 
# Start parallel processing

warnings.filterwarnings("ignore")
dask.compute(tasks)

### Combine Kerchunk Reference Files into Combined Reference

In [ ]:
%%time

# Create a list of reference json files
output_files = glob.glob(f"{target_store}/*.json")

# combine individual references into single consolidated reference
mzz = MultiZarrToZarr(
    output_files,
    concat_dims=["day"],
    identical_dims=["lat", "lon", "crs"],
)

multi_kerchunk = mzz.translate()

# Write in-memory Kerchunk combined reference to json
output_fname = "Combined_Reference.json"
with open(f"{output_fname}", "wb") as f:
    f.write(ujson.dumps(multi_kerchunk).encode())

## Load Kerchunk Dataset

In [ ]:
fs = fsspec.filesystem(
    "reference",
    fo="Combined_Reference.json",
    remote_protocol="http",
    remote_options={"anon": True},
    skip_instance_cache=True,
)
ds = xr.open_dataset(fs.get_mapper(""), engine="zarr")

In [ ]:
ds